In [1]:
import pandas as pd
import numpy as np

import re
import emoji
from soynlp.normalizer import repeat_normalize

import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset, TensorDataset

from transformers import T5ForConditionalGeneration, T5Tokenizer, pipeline
from transformers import ElectraTokenizerFast, ElectraModel, TFElectraModel
from transformers import AdamW

from sklearn.model_selection import train_test_split

In [20]:
# # 엑셀 파일 불러오기
# file_path = 'dataset.xlsx'
# df = pd.read_excel(file_path)

# # 피쳐와 타겟 추출
# X = df['사람문장1']
# y = df['감정_대분류']

# # 데이터 확인
# print(X.head())
# print(y.head())

In [2]:
file_path = 'kobertdataset.csv'
df = pd.read_csv(file_path)

X = df['발화문']
y = df['상황']

In [3]:
df['상황'].unique()

array(['happiness', 'neutral', 'sadness', 'angry', 'surprise', 'disgust',
       'fear'], dtype=object)

In [9]:
# emojis = ''.join(emoji.EMOJI_DATA.keys())
# pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣{emojis}]+')
# url_pattern = re.compile(
#     r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

# def clean(x): 
#     x = pattern.sub(' ', x)
#     x = emoji.replace_emoji(x, replace='')
#     x = url_pattern.sub('', x)
#     x = x.strip()
#     x = repeat_normalize(x, num_repeats=2)
#     return x

# X = X.apply(lambda x: clean(x))

In [10]:
# model = T5ForConditionalGeneration.from_pretrained('j5ng/et5-typos-corrector')
# tokenizer = T5Tokenizer.from_pretrained('j5ng/et5-typos-corrector')

# typos_corrector = pipeline(
#     "text2text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     device=0 if torch.cuda.is_available() else -1,
#     framework="pt",
# )

# X = X.apply(lambda x: typos_corrector(x,
#             max_length=128,
#             num_beams=5,
#             early_stopping=True)[0]['generated_text'])

In [11]:
# from transformers import ElectraTokenizerFast, AutoModelForSequenceClassification

# tokenizer = ElectraTokenizerFast.from_pretrained("kykim/electra-kor-base")
# model = AutoModelForSequenceClassification.from_pretrained("kykim/electra-kor-base", num_labels=6)

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")
# model = AutoModelForSequenceClassification.from_pretrained("beomi/KcELECTRA-base", num_labels=7)

In [13]:
# from transformers import BertTokenizerFast, AutoModelForSequenceClassification

# tokenizer = BertTokenizerFast.from_pretrained("kykim/gpt3-kor-small_based_on_gpt2")
# model = AutoModelForSequenceClassification.from_pretrained("kykim/gpt3-kor-small_based_on_gpt2", num_labels=6)
# model.config.pad_token_id = model.config.eos_token_id

In [14]:
# from transformers import FunnelTokenizerFast, AutoModelForSequenceClassification

# tokenizer = FunnelTokenizerFast.from_pretrained("kykim/funnel-kor-base")
# model = AutoModelForSequenceClassification.from_pretrained("kykim/funnel-kor-base", num_labels=6)

In [15]:
# from transformers import BertTokenizerFast, AutoModelForSequenceClassification

# tokenizer = BertTokenizerFast.from_pretrained("kykim/bert-kor-base")
# model = AutoModelForSequenceClassification.from_pretrained("kykim/bert-kor-base", num_labels=6)

In [16]:
# from transformers import ElectraTokenizerFast, AutoModelForSequenceClassification

# tokenizer = ElectraTokenizerFast.from_pretrained("monologg/koelectra-base-v3-discriminator")
# model = AutoModelForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels=6)

In [5]:
PATH = r'C:\Users\USER\Desktop\beomi.pth'
model = torch.load(PATH, map_location=torch.device('cpu'))

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

from sklearn.preprocessing import LabelEncoder

# Label encoding
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# 데이터 전처리 및 토큰화
X_list = X.values.tolist()
sequences = tokenizer(X_list, padding=True, truncation=True, return_tensors="pt")

X_train, X_test, y_train, y_test = train_test_split(sequences['input_ids'], y_encoded, test_size=0.2, random_state=42)
X_train_mask, X_test_mask, _, _ = train_test_split(sequences['attention_mask'], y_encoded, test_size=0.2, random_state=42)

# 텐서 변환
X_train, X_train_mask, y_train = X_train.to(device), X_train_mask.to(device), torch.tensor(y_train, dtype=torch.long).to(device)
X_test, X_test_mask, y_test = X_test.to(device), X_test_mask.to(device), torch.tensor(y_test, dtype=torch.long).to(device)

train_dataset = TensorDataset(X_train, X_train_mask, y_train)
test_dataset = TensorDataset(X_test, X_test_mask, y_test)

train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=16, shuffle=False)

optimizer = AdamW(model.parameters(), lr=3e-5)
criterion = nn.CrossEntropyLoss()

# 학습
EPOCHS = 5
for epoch in range(EPOCHS):
    # model.train()
    # for input_ids, attention_mask, labels in train_loader:
    #     input_ids = input_ids.to(device)
    #     attention_mask = attention_mask.to(device)
    #     labels = labels.to(device)
        
    #     optimizer.zero_grad()
    #     outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    #     logits = outputs.logits
    #     loss = criterion(logits, labels)
    #     loss.backward()
    #     optimizer.step()

    model.eval()
    val_losses = []
    val_accs = []
    for input_ids, attention_mask, labels in test_loader:
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_losses.append(outputs.loss.item())
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            acc = (preds == labels).float().mean().item()
            val_accs.append(acc)

    val_loss = np.mean(val_losses)
    val_acc = np.mean(val_accs)
    print(f"Epoch {epoch + 1}/{EPOCHS}, Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")

c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/5, Validation Loss: 0.30044952777205725, Validation Accuracy: 0.9197530864197531
Epoch 2/5, Validation Loss: 0.30044952777205725, Validation Accuracy: 0.9197530864197531
Epoch 3/5, Validation Loss: 0.30044952777205725, Validation Accuracy: 0.9197530864197531
Epoch 4/5, Validation Loss: 0.30044952777205725, Validation Accuracy: 0.9197530864197531
Epoch 5/5, Validation Loss: 0.30044952777205725, Validation Accuracy: 0.9197530864197531


In [25]:
torch.save(model, 'beomi.pth')

In [6]:
y_encoded

array([3, 3, 3, ..., 2, 3, 1])

In [7]:
y

0        happiness
1        happiness
2        happiness
3        happiness
4          neutral
           ...    
19369         fear
19370        angry
19371         fear
19372    happiness
19373      disgust
Name: 상황, Length: 19374, dtype: object

In [9]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

# 예시 레이블 리스트 (학습 시 사용한 레이블 리스트와 동일해야 합니다)
emotion_labels = ['happiness', 'neutral', 'sadness', 'angry', 'surprise', 'disgust', 'fear']

# LabelEncoder 인스턴스 생성 및 레이블 피팅
le = LabelEncoder()
le.fit(emotion_labels)

# 레이블과 숫자 매핑 출력
label_to_number = {label: idx for label, idx in zip(le.classes_, le.transform(le.classes_))}
number_to_label = {idx: label for label, idx in label_to_number.items()}

print("Label to Number Mapping:", label_to_number)
print("Number to Label Mapping:", number_to_label)


Label to Number Mapping: {'angry': 0, 'disgust': 1, 'fear': 2, 'happiness': 3, 'neutral': 4, 'sadness': 5, 'surprise': 6}
Number to Label Mapping: {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happiness', 4: 'neutral', 5: 'sadness', 6: 'surprise'}


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 라벨 인코딩
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# 데이터 전처리 및 토큰화
X_list = X.values.tolist()
sequences = tokenizer(X_list, padding=True, truncation=True, return_tensors="pt")

X_train, X_test, y_train, y_test = train_test_split(sequences['input_ids'], y_encoded, test_size=0.2, random_state=42)
X_train_mask, X_test_mask, _, _ = train_test_split(sequences['attention_mask'], y_encoded, test_size=0.2, random_state=42)

# 텐서 변환
X_train, X_train_mask, y_train = X_train.to(device), X_train_mask.to(device), torch.tensor(y_train, dtype=torch.long).to(device)
X_test, X_test_mask, y_test = X_test.to(device), X_test_mask.to(device), torch.tensor(y_test, dtype=torch.long).to(device)

train_dataset = TensorDataset(X_train, X_train_mask, y_train)
test_dataset = TensorDataset(X_test, X_test_mask, y_test)

train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=16, shuffle=False)

optimizer = AdamW(model.parameters(), lr=3e-5)
criterion = nn.CrossEntropyLoss()

# 학습
EPOCHS = 5
for epoch in range(EPOCHS):

    model.eval()
    val_losses = []
    val_accs = []
    all_preds = []
    all_labels = []
    
    for input_ids, attention_mask, labels in test_loader:
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_losses.append(outputs.loss.item())
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            acc = (preds == labels).float().mean().item()
            val_accs.append(acc)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_loss = np.mean(val_losses)
    val_acc = np.mean(val_accs)
    print(f"Epoch {epoch + 1}/{EPOCHS}, Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")

from sklearn.metrics import confusion_matrix, classification_report

# 클래스별 정확도 출력
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)
conf_matrix = confusion_matrix(all_labels, all_preds)
report = classification_report(all_labels, all_preds, target_names=le.classes_)
print(report)


c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/5, Validation Loss: 0.30044952777205725, Validation Accuracy: 0.9197530864197531
Epoch 2/5, Validation Loss: 0.30044952777205725, Validation Accuracy: 0.9197530864197531
Epoch 3/5, Validation Loss: 0.30044952777205725, Validation Accuracy: 0.9197530864197531
Epoch 4/5, Validation Loss: 0.30044952777205725, Validation Accuracy: 0.9197530864197531
Epoch 5/5, Validation Loss: 0.30044952777205725, Validation Accuracy: 0.9197530864197531
              precision    recall  f1-score   support

       angry       0.86      0.91      0.89       655
     disgust       0.93      0.88      0.90       465
        fear       0.94      0.95      0.95       281
   happiness       0.95      0.94      0.94       949
     neutral       0.91      0.93      0.92       634
     sadness       0.90      0.88      0.89       530
    surprise       0.97      0.95      0.96       361

    accuracy                           0.92      3875
   macro avg       0.92      0.92      0.92      3875
weighted avg 

In [11]:
(0.8811 + 0.9537 + 0.8796 + 0.9099) / 4

0.906075

In [12]:
0.9322

0.9322

In [13]:
0.9368

0.9368

In [10]:
# 클래스별 정확도 계산
class_accuracies = {}
for i, class_label in enumerate(le.classes_):
    true_positives = conf_matrix[i, i]
    total_samples = conf_matrix[i, :].sum()
    class_accuracies[class_label] = true_positives / total_samples

print("Class-wise Accuracy:")
for class_label, accuracy in class_accuracies.items():
    print(f"{class_label}: {accuracy:.4f}")

Class-wise Accuracy:
angry: 0.9099
disgust: 0.8796
fear: 0.9537
happiness: 0.9368
neutral: 0.9322
sadness: 0.8811
surprise: 0.9501
